# Train a GNN directly to an electric field

To execute this example fully, the following packages are required.

* openff-nagl
* openff-recharge
* openff-qcsubmit
* psi4

However, if you wish to just follow along the training part without first creating the training datasets yourself, you can get away with just `openff-nagl` installed and simply load the training/validation data from the provided `.parquet` files. The commands are provided at the end of the "Generate and format training data" section, but commented out.

In [5]:
import collections
import tqdm

from qcportal import PortalClient
from openff.units import unit

from openff.toolkit import Molecule, ForceField
from openff.qcsubmit.results import BasicResultCollection
from openff.recharge.esp.storage import MoleculeESPRecord
from openff.recharge.esp.qcresults import from_qcportal_results
from openff.recharge.grids import MSKGridSettings
from openff.recharge.utilities.geometry import compute_vector_field

import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

import torch
import MDAnalysis as mda

## Generate and format training data


### Downloading from QCArchive
First, we will create training data. We'll download a smaller training set for the purposes of this example.

In [2]:
qc_client = PortalClient("https://api.qcarchive.molssi.org:443", cache_dir=".")

# download dataset from QCPortal
br_esps_collection = BasicResultCollection.from_server(
    client=qc_client,
    datasets="OpenFF multi-Br ESP Fragment Conformers v1.1",
    spec_name="HF/6-31G*",
)

records_and_molecules = br_esps_collection.to_records()

### Convert to PyArrow dataset

NAGL reads in and trains to data from [PyArrow tables](https://arrow.apache.org/docs/python/getstarted.html#creating-arrays-and-tables). Below we do some conversion of each electric field to fit a basic GeneralLinearFit target, which fits the equation Ax = b. To avoid carrying too many data points around, we can do some postprocessing by first flattening the electric field matrix from 3 dimensions to 2, then multiplying by the transpose of A.

$$\mathbf{A}\vec{x} = \vec{b}$$
$$\mathbf{A^{T}}\mathbf{A}\vec{x} = \mathbf{A^{T}}\vec{b}$$

In [14]:
def calc_torsion_energy(angle, parameter):
    angle = (angle * unit.degrees).m_as(unit.radians)
    total = 0 * unit.kilojoules_per_mole
    for k, phase, periodicity in zip(parameter.k, parameter.phase, parameter.periodicity):
        phase = phase.m_as(unit.radians)
        subtotal = k * (1 + np.cos(periodicity * angle - phase))
        total += subtotal
    return total


def get_central_bond_torsions(mol, forcefield):
    labels = forcefield.label_molecules(mol.to_topology())[0]["ProperTorsions"]
    u = mda.Universe(mol.to_rdkit())

    bonds_to_dihedrals = collections.defaultdict(list)
    for key in labels:
        center = tuple(sorted(key[1:3]))
        bonds_to_dihedrals[center].append(key)

    # get energies in order of mol.bonds
    energies = []
    for bond in mol.bonds:
        key = tuple(sorted([bond.atom1_index, bond.atom2_index]))
        dihedrals = bonds_to_dihedrals[key]
        
        energy = 0 * unit.kilojoules_per_mole
        for dihedral_indices in dihedrals:
            dihedral_parameter = labels[dihedral_indices]
            angle = u.atoms[list(dihedral_indices)].dihedral.value()
            energy += calc_torsion_energy(angle, dihedral_parameter)

        energies.append(energy.m_as(unit.kilojoules_per_mole))
    return energies

In [15]:
pyarrow_entries = []
forcefield = ForceField("openff-2.1.0.offxml")
for record, molecule in tqdm.tqdm(records_and_molecules):
    central_torsion_energies = get_central_bond_torsions(molecule, forcefield)
    entry = {
        # hopefully this preserves bond order
        "mapped_smiles": molecule.to_smiles(mapped=True),
        "torsion_energies": central_torsion_energies
    }
    pyarrow_entries.append(entry)


# arbitrarily split into training and validation datasets
training_pyarrow_entries = pyarrow_entries[:-10]
validation_pyarrow_entries = pyarrow_entries[-10:]

training_table = pa.Table.from_pylist(training_pyarrow_entries)
validation_table = pa.Table.from_pylist(validation_pyarrow_entries)
training_table

100%|█████████████████████████████████████████| 650/650 [02:23<00:00,  4.53it/s]


pyarrow.Table
mapped_smiles: string
torsion_energies: list<item: double>
  child 0, item: double
----
mapped_smiles: [["[H:1][C:2](=[O:3])[C:4]1=[C:9]([C:7](=[N:6][S:5]1)[Br:8])[Br:10]","[H:1][C:2](=[O:3])[C:4]1=[C:5]([N:7]=[C:8]([N:10]1[H:11])[Br:9])[Br:6]","[H:1][C:2](=[O:3])[C:4]1=[C:10]([N:8]([C:6](=[N:5]1)[Br:7])[H:9])[Br:11]","[H:1][C:2](=[O:3])[C:4]1=[C:9]([S:8][N:7]=[C:5]1[Br:6])[Br:10]","[H:1][C:2](=[O:3])[C:4]1=[N:5][S:6][C:7](=[C:9]1[Br:10])[Br:8]",...,"[H:15][C:14]1([C:2]([O:4][C:5]([C:8]1([H:9])[C:10]([Br:11])([Br:12])[Br:13])([H:6])[H:7])([H:1])[H:3])[H:16]","[H:11][c:10]1[c:8]([c:6]([c:5]([c:14]([c:12]1[F:13])[Br:15])[C:3]([H:4])([C:16]([H:17])([H:18])[H:19])[O:2][H:1])[Br:7])[H:9]","[H:21][C:20]([H:22])([H:23])[N:19]([C:24]([H:25])([H:26])[H:27])[C:16]([H:17])([H:18])[C:13]([H:14])([H:15])[N+:10]([H:11])([H:12])[C:4]([H:5])([C:6]([Br:7])([Br:8])[Br:9])[N:2]([H:1])[H:3]","[H:21][C:20]([H:22])([H:23])[N:19]([C:24]([H:25])([H:26])[H:27])[C:16]([H:17])([H:18])[C:13]([H:14])

In [5]:
pq.write_table(training_table, "training_dataset_table.parquet")
pq.write_table(validation_table, "validation_dataset_table.parquet")

# # to read back in -- note, the files saved here give the full dataset, not the 50 record subset
# training_table = pq.read_table("training_dataset_table.parquet")
# validation_table = pq.read_table("validation_dataset_table.parquet")

## Set up for training a GNN

In [6]:
from openff.nagl.config import (
    TrainingConfig,
    OptimizerConfig,
    ModelConfig,
    DataConfig
)
from openff.nagl.config.model import (
    ConvolutionModule, ReadoutModule,
    ConvolutionLayer, ForwardLayer,
)
from openff.nagl.config.data import DatasetConfig
from openff.nagl.training.training import TrainingGNNModel
from openff.nagl.features.atoms import (
    AtomicElement,
    AtomConnectivity,
    AtomInRingOfSize,
    AtomAverageFormalCharge,
)

from openff.nagl.training.loss import GeneralLinearFitTarget

### Defining the training config

#### Defining a ModelConfig

First we define a ModelConfig.
This can be done in Python, but in practice it is probably easier to define the model in a YAML file and load it with `ModelConfig.from_yaml`.

In [7]:
atom_features = [
    AtomicElement(categories=["H", "C", "N", "O", "F", "Br", "S", "P", "I"]),
    AtomConnectivity(categories=[1, 2, 3, 4, 5, 6]),
    AtomInRingOfSize(ring_size=3),
    AtomInRingOfSize(ring_size=4),
    AtomInRingOfSize(ring_size=5),
    AtomInRingOfSize(ring_size=6),
    AtomAverageFormalCharge(),
]

# define our convolution module
convolution_module = ConvolutionModule(
    architecture="SAGEConv",
    # construct 6 layers with dropout 0 (default),
    # hidden feature size 512, and ReLU activation function
    # these layers can also be individually specified,
    # but we just duplicate the layer 6 times for identical layers
    layers=[
        ConvolutionLayer(
            hidden_feature_size=512,
            activation_function="ReLU",
            aggregator_type="mean"
        )
    ] * 6,
)

# define our readout module/s
# multiple are allowed but let's focus on charges
readout_modules = {
    # key is the name of output property, any naming is allowed
    "charges": ReadoutModule(
        pooling="atoms",
        postprocess="compute_partial_charges",
        # 2 layers
        layers=[
            ForwardLayer(
                hidden_feature_size=512,
                activation_function="ReLU",
            )
        ] * 2,
    )
}

# bring it all together
model_config = ModelConfig(
    version="0.1",
    atom_features=atom_features,
    convolution=convolution_module,
    readouts=readout_modules,
)

#### Defining a DataConfig

We can then define our dataset configs. Here we also have to specify our training targets.

In [8]:
target = GeneralLinearFitTarget(
    # what we're using to evaluate loss
    target_label="prediction_vector",
    # the output of the GNN we use to evaluate loss
    prediction_label="charges",
    # the column in the table that contains the precursor matrix
    design_matrix_column="precursor_matrix",
    # how we want to evaluate loss, e.g. RMSE, MSE, ...
    metric="rmse",
    # how much to weight this target
    # helps with scaling in multi-target optimizations
    weight=1,
    denominator=1,
)

training_to_electric_field = DatasetConfig(
    sources=["training_dataset_table.parquet"],
    targets=[target],
    batch_size=100,
)
validating_to_electric_field = DatasetConfig(
    sources=["validation_dataset_table.parquet"],
    targets=[target],
    batch_size=100,
)

# bringing it together
data_config = DataConfig(
    training=training_to_electric_field,
    validation=validating_to_electric_field
)

#### Defining an OptimizerConfig

The optimizer config is relatively simple; the only moving part here currently is the learning rate.

In [9]:
optimizer_config = OptimizerConfig(optimizer="Adam", learning_rate=0.001)

#### Creating a TrainingConfig

In [10]:
training_config = TrainingConfig(
    model=model_config,
    data=data_config,
    optimizer=optimizer_config
)

### Creating a TrainingGNNModel

Now we can create a `TrainingGNNModel`, which allows easy training of a `GNNModel`. The `GNNModel` can be accessed through `TrainingGNNModel.model`.

In [11]:
training_model = TrainingGNNModel(training_config)
training_model

TrainingGNNModel(
  (model): GNNModel(
    (convolution_module): ConvolutionModule(
      (gcn_layers): SAGEConvStack(
        (0): SAGEConv(
          (feat_drop): Dropout(p=0.0, inplace=False)
          (activation): ReLU()
          (fc_neigh): Linear(in_features=20, out_features=512, bias=False)
          (fc_self): Linear(in_features=20, out_features=512, bias=True)
        )
        (1-5): 5 x SAGEConv(
          (feat_drop): Dropout(p=0.0, inplace=False)
          (activation): ReLU()
          (fc_neigh): Linear(in_features=512, out_features=512, bias=False)
          (fc_self): Linear(in_features=512, out_features=512, bias=True)
        )
      )
    )
    (readout_modules): ModuleDict(
      (charges): ReadoutModule(
        (pooling_layer): PoolAtomFeatures()
        (readout_layers): SequentialLayers(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=512, 

We can look at the initial capabilities of the model by comparing its charges to AM1-BCC charges. They're pretty bad.

In [12]:
test_molecule = Molecule.from_smiles("CCCBr")
test_molecule.assign_partial_charges("am1bcc")
reference_charges = test_molecule.partial_charges.m

# switch to eval mode
training_model.model.eval()

with torch.no_grad():
    nagl_charges_1 = training_model.model.compute_properties(
        test_molecule,
        as_numpy=True
    )["charges"]

# switch back to training mode
training_model.model.train()

# compare charges
differences = reference_charges - nagl_charges_1
differences

array([-0.0422502 , -0.05428261,  0.15142802, -0.42682151,  0.09391647,
        0.09391647,  0.09391647,  0.2050635 ,  0.2050635 , -0.15997499,
       -0.15997499])

### Training the model

We use Pytorch Lightning to train.

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import TQDMProgressBar

In [ ]:
trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[TQDMProgressBar()], # add progress bar
    accelerator="cpu"
)

In [ ]:
datamodule = training_model.create_data_module(verbose=False)

In [14]:
trainer.fit(
    training_model,
    datamodule=datamodule,
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/lily/micromamba/envs/train-gnn-to-electric-field/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Featurizing dataset: 0it [00:00, ?it/s]
Featurizing batch: 100%|████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 159.17it/s]
Featurizing dataset: 1it [00:00,  3.81it/s]
Featurizing dataset: 0it [00:00, ?it/s]
Featurizing batch: 100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 176.62it/s]
Featurizing dataset: 1it [00:00, 16.94it/s]
Featurizing dataset: 0it [00:00, ?it/s]
Featurizing batch: 100%|████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 182.97it/s]
Featurizing dataset: 1it [00:00,  4.46it/s]
Featurizing

Sanity Checking: |                                                                                            …

/Users/lily/micromamba/envs/train-gnn-to-electric-field/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/lily/micromamba/envs/train-gnn-to-electric-field/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/lily/micromamba/envs/train-gnn-to-electric-field/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training 

Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=100` reached.


We can now check the charges again. They should have improved, especially if you used a larger dataset (note: results may vary with the small 50-molecule dataset this notebook has chosen to use, for reasons of speed).

In [15]:
# switch to eval mode
training_model.model.eval()

with torch.no_grad():
    nagl_charges_2 = training_model.model.compute_properties(
        test_molecule,
        as_numpy=True
    )["charges"]

differences_after_training = reference_charges - nagl_charges_2
differences_after_training

array([-0.10230289, -0.0789829 , -0.01655325, -0.16298471,  0.03418326,
        0.03418326,  0.03418326,  0.05784283,  0.05784283,  0.07129431,
        0.07129431])